# **Harshit Singh **

---






```
#please insert  your openai api key and install the dependencies before running the file.
```




---




## Setup

In [ ]:
%%capture
!pip install openai langchain  tiktoken pypdf unstructured[local-inference] gradio chromadb

In [ ]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone, Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

In [ ]:
os.environ['OPENAI_API_KEY'] =""

In [ ]:
#llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.document_loaders import DirectoryLoader

pdf_loader = DirectoryLoader('/content/drive/MyDrive/LegalData', glob="**/*.pdf")
readme_loader = DirectoryLoader('/content/drive/MyDrive/LegalData', glob="**/*.md")
txt_loader = DirectoryLoader('/content/drive/MyDrive/LegalData', glob="**/*.txt")

In [ ]:
#take all the loader
loaders = [pdf_loader, readme_loader, txt_loader]

#lets create document
documents = []
for loader in loaders:
    documents.extend(loader.load())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
print (f'You have {len(documents)} document(s) in your data')
print (f'There are {len(documents[0].page_content)} characters in your document')

You have 13 document(s) in your data
There are 40795 characters in your document


In [ ]:
documents[0]

Document(page_content='THE MARINE AIDS TO NAVIGATION ACT, 2021 ______________\n\nARRANGEMENT OF SECTIONS ______________\n\nCHAPTER I\n\nPRELIMINARY\n\nSECTIONS\n\n1. Short title, extent and commencement. 2. Definitions.\n\nCHAPTER II\n\nDESIGNATION OF GENERAL AID TO NAVIGATION\n\n3. Power to designate general aid to navigation.\n\nCHAPTER III\n\nDIRECTOR GENERAL OF AIDS TO NAVIGATION\n\n4. Appointment of Director General, Deputy Director Generals and Directors. 5. Duties of Director General. 6. Central Advisory Committee. 7. Proceedings of Central Advisory Committee not to be invalidated.\n\nCHAPTER IV\n\nMANAGEMENT OF GENERAL AIDS TO NAVIGATION\n\n8. Management of general aids to navigation. 9. Powers of Central Government relating to aids to navigation.\n\nCHAPTER V\n\nMANAGEMENT OF VESSEL TRAFFIC SERVICES\n\n10. Management of vessel traffic services. 11. Powers of Central Government relating to vessel traffic services. 12. Competent Authority for Vessel Traffic Services. 13. Standar

## Split the Text from the documents

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=40) #chunk overlap seems to work better
documents = text_splitter.split_documents(documents)
print(len(documents))

695


In [ ]:
documents[0]

Document(page_content='THE MARINE AIDS TO NAVIGATION ACT, 2021 ______________\n\nARRANGEMENT OF SECTIONS ______________\n\nCHAPTER I\n\nPRELIMINARY\n\nSECTIONS\n\n1. Short title, extent and commencement. 2. Definitions.\n\nCHAPTER II\n\nDESIGNATION OF GENERAL AID TO NAVIGATION\n\n3. Power to designate general aid to navigation.\n\nCHAPTER III\n\nDIRECTOR GENERAL OF AIDS TO NAVIGATION\n\n4. Appointment of Director General, Deputy Director Generals and Directors. 5. Duties of Director General. 6. Central Advisory Committee. 7. Proceedings of Central Advisory Committee not to be invalidated.\n\nCHAPTER IV\n\nMANAGEMENT OF GENERAL AIDS TO NAVIGATION\n\n8. Management of general aids to navigation. 9. Powers of Central Government relating to aids to navigation.\n\nCHAPTER V\n\nMANAGEMENT OF VESSEL TRAFFIC SERVICES', metadata={'source': '/content/drive/MyDrive/LegalData/A2021-20.pdf'})

In [ ]:
documents[1]

Document(page_content='MANAGEMENT OF VESSEL TRAFFIC SERVICES\n\n10. Management of vessel traffic services. 11. Powers of Central Government relating to vessel traffic services. 12. Competent Authority for Vessel Traffic Services. 13. Standards for establishment and operation of vessel traffic services.\n\nCHAPTER VI\n\nINSPECTION AND MANAGEMENT OF LOCAL AIDS TO NAVIGATION\n\n14. Power to inspect local aids to navigation. 15. Control of local aids to navigation by Central Government 16. Management of local aids to navigation by Central Government.\n\n1\n\nSECTIONS\n\nCHAPTER VII\n\nOBSTRUCTION TO FUNCTIONING OF AIDS TO NAVIGATION\n\n17. Power of Central Government to remove or alter obstructions to aids to navigation.\n\nCHAPTER VIII\n\nTRAINING AND CERTIFICATION\n\n18. Power of Central Government to train and certify operators of aids to navigation and vessel traffic\n\nservices. 19. Certification. 20. Accreditation of training organisations.\n\nCHAPTER IX\n\nMARKING OF WRECKS\n\n21. M

## Embeddings and storing it in Vectorestore

In [ ]:
embeddings = OpenAIEmbeddings()

### Using Chroma for storing vectors

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
vectorstore = Chroma.from_documents(documents, embeddings)

In [ ]:
from langchain.llms import OpenAI

In [ ]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":4})

qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.5), retriever)

In [ ]:
chat_history = []
query = "what is the law for criminals"



# Using similarity search to find out the similarity bw different diocuments and the query -- basically the distances of vector embeddings

In [ ]:
docs = vectorstore.similarity_search_with_score(query)

In [ ]:
docs

[(Document(page_content='10\n\nProvided that nothing contained in this sub-section shall render any such person liable to any punishment provided in this Act, if he proves that the offence was committed without his knowledge or that he exercised all due diligence to prevent the commission of such offence.\n\n(4) Notwithstanding anything contained in sub-section (3), where an offence under this Act has been committed by a company and it is proved that the offence has been committed with the consent or connivance of, or is attributable to any neglect on the part of, any director, manager, secretary or other officer of the company, such director, manager, secretary or other officer shall also be deemed to be guilty of that offence and shall be liable to be proceeded against and punished accordingly.\n\nExplanation.—For the purposes of sub-sections (3) and (4),—\n\n(a) "company" means any body corporate, and includes a firm or other association of individuals;\n\nand', metadata={'source': 

# Extracting the most relevants document's sources and their scores

In [ ]:
# Creating list to store details
details_list = []

# Extract similarity score, source, page number, and page content for each document and store them in the list
for doc, similarity_score in docs:
    page_content = doc.page_content
    metadata = doc.metadata

    # Extract page number
    page_number = page_content.split('\n')[0]

    # Extract source
    source = metadata.get('source', 'Source information not available')

    # Append the details as a tuple to the list, including page content
    details_list.append((similarity_score, source, page_number, page_content))

# Sort the list in descending order based on similarity score
details_list.sort(reverse=True)

# Print the sorted details
for similarity_score, source, page_number, page_content in details_list:
    print("Similarity Score:", similarity_score)
   # print("Page Number:", page_number)
    print("Source:", source)
    #print("Page Content:", page_content)
    print("\n")  # Add a separator for clarity



Similarity Score: 0.4434756636619568
Source: /content/drive/MyDrive/LegalData/A2021-20.pdf


Similarity Score: 0.44111090898513794
Source: /content/drive/MyDrive/LegalData/A2021-42 .pdf


Similarity Score: 0.44064393639564514
Source: /content/drive/MyDrive/LegalData/A2021-20.pdf


Similarity Score: 0.43111133575439453
Source: /content/drive/MyDrive/LegalData/a2021-29.pdf




# Only the top similarity score documents are fed to the chain for generating the responses

In [ ]:
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' Under this Act, all offences punishable under this Act are cognizable and non-bailable. For offences under section 7, 9, 10, 17, the punishment is imprisonment for a term which may extend to seven years and with fine which shall not be less than ten lakh rupees but which may extend to fifty lakh rupees. For offences under section 17, the punishment is an imprisonment for a term which shall not be less than twenty years but which may extend to imprisonment for life and with fine which shall not be less than fifty crore rupees for any nuclear explosion in Antarctica, or an imprisonment for a term which shall not be less than fourteen years but which may extend to imprisonment for life and with fine which shall not be less than twenty-five crore rupees for disposal of any radioactive waste material in Antarctica.'

In [ ]:
data=retriever.get_relevant_documents(query)

In [ ]:
# List of document objects
documents = data

# Iterate through the documents and extract information
for document in documents:
    page_content = document.page_content
    metadata = document.metadata

    # Extract page number (assuming it's the number at the beginning of the content)
    page_number = page_content.split('\n')[0]

    # Extract source
    source = metadata.get('source', 'Source information not available')

    # Print the extracted information for each document
    print("Page Number:", page_number)
    print("Source:", source)
    print("\n")  # Add a separator for clarity


Page Number: 10
Source: /content/drive/MyDrive/LegalData/a2021-29.pdf


Page Number: adoption or through surrogacy earlier:
Source: /content/drive/MyDrive/LegalData/A2021-47.pdf


Page Number: (d) Ommatophocarossii (Ross Seal) or any other species as may be prescribed shall be accorded special protection and permit for killing, injuring, capturing or handling of these species may be issued only for scientific purpose, if the survival or recovery of that species or local population is not jeopardised, and non-lethal techniques are used as far as possible; and
Source: /content/drive/MyDrive/LegalData/a2022-13.pdf


Page Number: 67. Application of other laws not barred.—The provisions of this Act shall be in addition to and
Source: /content/drive/MyDrive/LegalData/A2021-1.pdf




In [ ]:
chat_history.append((query, result["answer"]))
chat_history

[('what is the law for criminals',
  ' Under this Act, all offences punishable under this Act are cognizable and non-bailable. For offences under section 7, 9, 10, 17, the punishment is imprisonment for a term which may extend to seven years and with fine which shall not be less than ten lakh rupees but which may extend to fifty lakh rupees. For offences under section 17, the punishment is an imprisonment for a term which shall not be less than twenty years but which may extend to imprisonment for life and with fine which shall not be less than fifty crore rupees for any nuclear explosion in Antarctica, or an imprisonment for a term which shall not be less than fourteen years but which may extend to imprisonment for life and with fine which shall not be less than twenty-five crore rupees for disposal of any radioactive waste material in Antarctica.')]

In [ ]:
query = "Act not intended and not known to be likely to cause death or grievous hurt, done by consent."
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' There is no specific punishment provided for offences under the Act which are not intended and not known to be likely to cause death or grievous hurt, done by consent. If the offence is committed by a company, the director, manager, secretary, or other officer of the company may be deemed guilty of the offence and liable to be proceeded against and punished accordingly.'